# PydanticAI fundamentals

In [1]:
from pydantic_ai import Agent

agent = Agent(model = "google-gla:gemini-2.5-flash")

result = await agent.run("Give me an IT employee working in Sweden, no yapping")

result

AgentRunResult(output='**Name:** Linnea Karlsson\n**Role:** Software Developer\n**Location:** Stockholm, Sweden\n**Focus:** Backend systems (Java, Spring Boot), cloud infrastructure (AWS).')

In [3]:
print(result.output)

**Name:** Linnea Karlsson
**Role:** Software Developer
**Location:** Stockholm, Sweden
**Focus:** Backend systems (Java, Spring Boot), cloud infrastructure (AWS).


## use pydantic to structure output

In [6]:
from pydantic import BaseModel, Field

class Employee(BaseModel):
    name: str = Field(description="first name and last name of the person")
    age: int = Field(gt=17, lt=67, description="age of a person")
    salary: int = Field(gt=30_000, lt=70_000, description="salary of the employee, the more senior the role, the more salary")
    role: str = Field(description="always include seniority level in the role (junior, senior, expert)")
    description: str = Field(description="short description (max 2 sentences) of the role, include keywords")

result = await agent.run("Give me an IT employee working as Data Engineer in Sweden", output_type=Employee)

result

AgentRunResult(output=Employee(name='Erik Svensson', age=35, salary=65000, role='Senior Data Engineer', description='Designs, builds, and maintains scalable data pipelines and ETL processes. Focuses on optimizing data flow and architecture for large datasets.'))

In [7]:
employee = result.output
employee

Employee(name='Erik Svensson', age=35, salary=65000, role='Senior Data Engineer', description='Designs, builds, and maintains scalable data pipelines and ETL processes. Focuses on optimizing data flow and architecture for large datasets.')

In [8]:
employee.description, employee.name

('Designs, builds, and maintains scalable data pipelines and ETL processes. Focuses on optimizing data flow and architecture for large datasets.',
 'Erik Svensson')

In [9]:
isinstance(employee, BaseModel)

True

In [11]:
employee.model_dump()

{'name': 'Erik Svensson',
 'age': 35,
 'salary': 65000,
 'role': 'Senior Data Engineer',
 'description': 'Designs, builds, and maintains scalable data pipelines and ETL processes. Focuses on optimizing data flow and architecture for large datasets.'}

In [12]:
employee.model_dump_json()

'{"name":"Erik Svensson","age":35,"salary":65000,"role":"Senior Data Engineer","description":"Designs, builds, and maintains scalable data pipelines and ETL processes. Focuses on optimizing data flow and architecture for large datasets."}'

In [13]:
print(employee.model_dump_json(indent=3))

{
   "name": "Erik Svensson",
   "age": 35,
   "salary": 65000,
   "role": "Senior Data Engineer",
   "description": "Designs, builds, and maintains scalable data pipelines and ETL processes. Focuses on optimizing data flow and architecture for large datasets."
}


## simulate 10 employees

In [15]:
result = await agent.run(
    "Give 10 employees in AI, data science, data engineering, data analyst field",
    output_type=list[Employee]
    )

In [17]:
for employee in result.output:
    print(f"{employee.name} is {employee.age} years old, earns {employee.salary} and have role {employee.role}")

Alice Smith is 30 years old, earns 65000 and have role Senior AI Engineer
Bob Johnson is 35 years old, earns 68000 and have role Expert Data Scientist
Charlie Brown is 28 years old, earns 55000 and have role Junior Data Engineer
Diana Prince is 40 years old, earns 69000 and have role Expert AI Architect
Ethan Hunt is 32 years old, earns 60000 and have role Senior Data Analyst
Fiona Glenanne is 25 years old, earns 50000 and have role Junior Data Scientist
George Costanza is 38 years old, earns 67000 and have role Senior Data Engineer
Hannah Montana is 29 years old, earns 58000 and have role AI Engineer
Ivy League is 33 years old, earns 63000 and have role Senior Data Scientist
Jack Sparrow is 27 years old, earns 52000 and have role Junior Data Analyst


## More complex models

In [20]:
from typing import Literal

class Department(BaseModel):
    name: Literal["IT", "HR", "Marketing"] = Field(description="name of the department")
    description: str = Field(description="short description of the department")
    employees: list[Employee]


class Company(BaseModel):
    name: str = Field(description="name of the company")
    departments: list[Department]
    description: str = Field(description="describe company in two sentences")


result = await agent.run(
    ("Give 2-3 employees in each department",
    "Company should be called CoolCompany and sell ice creams"),
    output_type=Company
    )

In [23]:
result.output

Company(name='CoolCompany', departments=[Department(name='IT', description='The IT department manages all technology infrastructure and software development for CoolCompany.', employees=[Employee(name='Alice Smith', age=30, salary=65000, role='Junior Software Developer', description='Alice is responsible for developing and maintaining internal software systems. She works with Python and JavaScript.'), Employee(name='Bob Johnson', age=45, salary=69000, role='Senior IT Manager', description='Bob oversees the IT department operations and infrastructure. He ensures all systems run smoothly and securely.')]), Department(name='HR', description='The HR department handles all human resources aspects, including recruitment, employee relations, and talent management.', employees=[Employee(name='Carol White', age=35, salary=60000, role='HR Specialist', description='Carol handles recruitment and employee relations. She ensures a positive working environment.'), Employee(name='David Green', age=50,

In [22]:
company = result.output
company.__dict__.keys()

dict_keys(['name', 'departments', 'description'])

In [24]:
company.name

'CoolCompany'

In [25]:
[department.name for department in company.departments]

['IT', 'HR', 'Marketing']

In [30]:
departments = [department for department in company.departments]
departments[0].employees

[Employee(name='Alice Smith', age=30, salary=65000, role='Junior Software Developer', description='Alice is responsible for developing and maintaining internal software systems. She works with Python and JavaScript.'),
 Employee(name='Bob Johnson', age=45, salary=69000, role='Senior IT Manager', description='Bob oversees the IT department operations and infrastructure. He ensures all systems run smoothly and securely.')]

In [27]:
company.model_dump()

{'name': 'CoolCompany',
 'departments': [{'name': 'IT',
   'description': 'The IT department manages all technology infrastructure and software development for CoolCompany.',
   'employees': [{'name': 'Alice Smith',
     'age': 30,
     'salary': 65000,
     'role': 'Junior Software Developer',
     'description': 'Alice is responsible for developing and maintaining internal software systems. She works with Python and JavaScript.'},
    {'name': 'Bob Johnson',
     'age': 45,
     'salary': 69000,
     'role': 'Senior IT Manager',
     'description': 'Bob oversees the IT department operations and infrastructure. He ensures all systems run smoothly and securely.'}]},
  {'name': 'HR',
   'description': 'The HR department handles all human resources aspects, including recruitment, employee relations, and talent management.',
   'employees': [{'name': 'Carol White',
     'age': 35,
     'salary': 60000,
     'role': 'HR Specialist',
     'description': 'Carol handles recruitment and employ